In [ ]:
import pandas as pd
import numpy as np

college = pd.read_csv(r'college-salaries/salaries-by-college-type.csv')

# NOTEBOOK CREATED BY: Vincent Lao
# https://www.kaggle.com/wsj/college-salaries

In [ ]:
college.head(5)

In [ ]:
college.isnull().sum()

In [ ]:
college.shape

In [ ]:
region = pd.read_csv(r'college-salaries/salaries-by-region.csv')
region.head()

In [ ]:
region.isnull().sum()

In [ ]:
region.shape

In [ ]:
region["Mid-Career 10th Percentile Salary"].head()

In [ ]:
college[college['School Name'].str.contains('University of California')]
# choose the indices (rows) from college where the 'School Name' contains the string 'University of California'

In [ ]:
cols = ["School Name", "Starting Median Salary", "Mid-Career Median Salary", "Mid-Career 10th Percentile Salary", "Mid-Career 25th Percentile Salary", "Mid-Career 75th Percentile Salary", "Mid-Career 90th Percentile Salary"]
combined = college.merge(region, how='outer', on=cols)
combined.head()

In [ ]:
combined.isnull().sum()

In [ ]:
combined.shape

In [ ]:
missing = combined[combined["Mid-Career 10th Percentile Salary"].isnull()]
missing.head()

In [ ]:
# If 25th percentile == x, and 75th percentile == y, and we assume that it's roughly normal, then we can find a rough mean,
# and use z-score to calculate the 10th and 90th percentile to fill in our data.
missing.where(missing["Mid-Career 10th Percentile Salary"].isnull()).index \
== missing.where(missing["Mid-Career 90th Percentile Salary"].isnull()).index

In [ ]:
s# Any guesses for the median starting salaries?
np.mean(missing["Mid-Career 25th Percentile Salary"])

### It errors! Why?
If you look closely, the data is actually strings that start with a dollar sign and includes commas. Without thinking too much about it, we don't realize that the commas and dollar signs are even there -- it's just intuitive for us, but the computer sees it. Let's fix it.

In [ ]:
missing["Mid-Career 25th Percentile Salary"] = missing["Mid-Career 25th Percentile Salary"].copy().str.replace(r"\$|,", "").astype(float);
missing["Mid-Career 75th Percentile Salary"] = missing["Mid-Career 75th Percentile Salary"].copy().str.replace(r"\$|,", "").astype(float);

In [ ]:
mus = (missing["Mid-Career 25th Percentile Salary"] + missing["Mid-Career 75th Percentile Salary"]) / 2
mus.head()

In [ ]:
with_mus = missing.copy()
with_mus["Mid-Career 50th Percentile Salary"] = mus
with_mus.head()

In [ ]:
# If 25th percentile == x, and 75th percentile == y, and we assume that it's roughly normal, then we can find a rough mean,
# and use z-score to calculate the 10th and 90th percentile to fill in our data.
z_10 = -1.282
z_25 = -0.675
z_75 = 0.675
z_90 = 1.282

# Z = (X - mu) / sigma; ex: for Caltech, z_25 = (123,000 - 132500) / sigma; sigma = (123,000 - 132,500) / -0.675
sigmas_from_25 = (with_mus["Mid-Career 25th Percentile Salary"] -  with_mus["Mid-Career 50th Percentile Salary"]) / -0.675
sigmas_from_25.head()

In [ ]:
# Check that we get the same sigmas from
sigmas_from_75 = (with_mus["Mid-Career 25th Percentile Salary"] -  with_mus["Mid-Career 50th Percentile Salary"]) / -0.675
sigmas_from_75.head()

In [ ]:
with_mus_sds = with_mus.copy()
with_mus_sds["Mid-Career SD"] = sigmas_from_25
with_mus_sds.head()

In [ ]:
# Z_10 = (X_10 - mu) / sigma; X_10 = Z_10 * sigma + mu
filled = with_mus_sds.copy()
filled["Mid-Career 10th Percentile Salary"] = z_10 * filled["Mid-Career SD"] + filled["Mid-Career 50th Percentile Salary"]

# Z_90 = (X_90 - mu) / sigma; X_90 = Z_90 * sigma + mu
filled["Mid-Career 90th Percentile Salary"] = z_90 * filled["Mid-Career SD"] + filled["Mid-Career 50th Percentile Salary"]

In [ ]:
filled.head()

In [ ]:
final = filled.merge(combined, how="left", on=cols + ["School Type", "Region"]).head()
final.head()

In [ ]:
final.isnull().sum()

Good job getting through this document!
There's plenty of other ways to clean data, but here's just a couple to start off :)
Ask your mentors questions if you are confused!
-Vincent Lao